In [ ]:
pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 28.4 MB/s 
     |████████████████████████████████| 401 kB 71.3 MB/s 
     |████████████████████████████████| 5.8 MB 59.8 MB/s 
     |████████████████████████████████| 3.6 MB 52.4 MB/s 
     |████████████████████████████████| 60 kB 9.3 MB/s 
     |████████████████████████████████| 125 kB 86.7 MB/s 
     |████████████████████████████████| 41.4 MB 1.3 MB/s 
     |████████████████████████████████| 769 kB 69.4 MB/s 
     |████████████████████████████████| 365 kB 77.0 MB/s 
     |████████████████████████████████| 95 kB 6.6 MB/s 
     |████████████████████████████████| 182 kB 81.2 MB/s 
     |████████████████████████████████| 132 kB 77.3 MB/s 
     |████████████████████████████████| 212 kB 79.0 MB/s 
     |████████████████████████████████| 127 kB 68.9 MB/s 
     |████████████████████████████████| 7.6 MB 59.4 MB/s 
     |█████████████████████████████

In [ ]:
import os
import pandas as pd
import numpy as np
from skimage.io import imread, imshow
from skimage import filters
from skimage import feature
from skimage.filters import prewitt_h,prewitt_v
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pickle
import re
import warnings
warnings.filterwarnings("ignore")
from textattack.augmentation import WordNetAugmenter
from textattack.augmentation import EasyDataAugmenter

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!gdown 19xH2647kd7hVO7GcpuLDzuqZ9_UelHuq
# https://drive.google.com/file/d/19xH2647kd7hVO7GcpuLDzuqZ9_UelHuq/view?usp=share_link

Downloading...
From: https://drive.google.com/uc?id=19xH2647kd7hVO7GcpuLDzuqZ9_UelHuq
To: /content/final_end.csv
100% 468k/468k [00:00<00:00, 163MB/s]


In [ ]:
df = pd.read_csv("final_end.csv")
df.head(5)

,Unnamed: 0,text_corrected,overall_sentiment
0,0,look friend lighter sohalikut trend play years...,very_positive
1,1,best yearchallenge completed less years lupus ...,very_positive
2,2,sam throne stripping follow saw everyone posti...,positive
3,3,year challenge sweet see edition,positive
4,4,year challenge filter hilarious metes,neutral


In [ ]:
s={'very_positive':1,'positive':1, 'neutral':0,'negative':-1,'very_negative':-1}
df["overall_sentiment"]=df["overall_sentiment"].map(s)
df

,Unnamed: 0,text_corrected,overall_sentiment
0,0,look friend lighter sohalikut trend play years...,1
1,1,best yearchallenge completed less years lupus ...,1
2,2,sam throne stripping follow saw everyone posti...,1
3,3,year challenge sweet see edition,1
4,4,year challenge filter hilarious metes,0
...,...,...,...
6981,6987,tuesday made gras wednesday valentine friday c...,0
6982,6988,must watch moves chennai metes maanagaram hand...,0
6983,6989,less talking planning soda june food complaini...,1
6984,6990,time fantasy one unless make arhtisticlicensecom,1


In [ ]:
correct = df.loc[df['overall_sentiment'] == -1]
correct

,Unnamed: 0,text_corrected,overall_sentiment
5,5,dont get car strangers meet people internet al...,-1
6,6,years challenge lit b berg transformation,-1
10,10,probably first man year challenge chennai pasa...,-1
14,14,antivaxx kiss see someone year challenge,-1
49,49,hawkers important infinity war though,-1
...,...,...,...
6954,6960,face friend gives crush walks room,-1
6955,6961,nightmare made,-1
6959,6965,baby mr bean funny picture,-1
6965,6971,face memecentercom substitute path class,-1


In [ ]:
# wordnet_aug = WordNetAugmenter()
eda_aug = EasyDataAugmenter()
new = []

for i in range(len(correct.index.values)):
  rephrased = eda_aug.augment(str(correct.iloc[i,1]))
  new.append(rephrased)


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
temp = df.drop(["Unnamed: 0"],axis=1).copy(deep=True)
temp

,text_corrected,overall_sentiment
0,look friend lighter sohalikut trend play years...,1
1,best yearchallenge completed less years lupus ...,1
2,sam throne stripping follow saw everyone posti...,1
3,year challenge sweet see edition,1
4,year challenge filter hilarious metes,0
...,...,...
6981,tuesday made gras wednesday valentine friday c...,0
6982,must watch moves chennai metes maanagaram hand...,0
6983,less talking planning soda june food complaini...,1
6984,time fantasy one unless make arhtisticlicensecom,1


In [ ]:
for i in range(len(new)):
  temp = temp.append({"text_corrected": new[i][0],"overall_sentiment" : -1},ignore_index=True)
temp

,text_corrected,overall_sentiment
0,look friend lighter sohalikut trend play years...,1
1,best yearchallenge completed less years lupus ...,1
2,sam throne stripping follow saw everyone posti...,1
3,year challenge sweet see edition,1
4,year challenge filter hilarious metes,0
...,...,...
7612,gives friend face crush walks room,-1
7613,nightmare,-1
7614,baby bean mr funny picture,-1
7615,face way memecentercom substitute path class,-1


In [ ]:
temp.to_csv('finale_text.csv')

In [ ]:
df.shape

(6986, 3)

In [ ]:
temp.shape

(7617, 2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
temp = temp.dropna(subset=["text_corrected"])
vectorizer = TfidfVectorizer(lowercase=True)
X = vectorizer.fit_transform(temp['text_corrected'])
y = temp["overall_sentiment"]

In [ ]:
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
df.tail(5)

,3rd,aaaaaaaaa,aaaaaaaaaaand,aaaaaaaahhh,aaaaaaaand,aaaand,aaaargh,aah,aaron,aasldfliksdfjalsdfkis,...,zone,zubova,zuckerberg,zuckerbergexe,zuckerbergmemes,zuckerbergts,zuckerburg,zuckerphobia,zukerberg,zxcvbnm
7603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.iloc[len(df.index.values)-1]

3rd              0.0
aaaaaaaaa        0.0
aaaaaaaaaaand    0.0
aaaaaaaahhh      0.0
aaaaaaaand       0.0
                ... 
zuckerbergts     0.0
zuckerburg       0.0
zuckerphobia     0.0
zukerberg        0.0
zxcvbnm          0.0
Name: 7607, Length: 9961, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,shuffle=True)
print('Shape of X_train = ', X_train.shape)
print('Shape of y_train = ', y_train.shape)
print('Shape of X_test = ', X_test.shape)
print('Shape of y_test = ', y_test.shape)
y_train.value_counts()

Shape of X_train =  (5325, 9978)
Shape of y_train =  (5325,)
Shape of X_test =  (2283, 9978)
Shape of y_test =  (2283,)


 1    2906
 0    1551
-1     868
Name: overall_sentiment, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=15)
clf.fit(X_train, y_train)


RandomForestClassifier(n_estimators=15)

In [ ]:
pred=clf.predict(X_test)
clf.score(X_test,y_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test,pred))
print("F1 Score : ",f1_score(y_test, pred, average='macro')*100)
print(confusion_matrix(y_test,pred))

Accuracy :  58.86990801576872
Report : 
               precision    recall  f1-score   support

          -1       0.80      0.54      0.64       393
           0       0.33      0.15      0.21       647
           1       0.60      0.83      0.70      1243

    accuracy                           0.59      2283
   macro avg       0.58      0.51      0.52      2283
weighted avg       0.56      0.59      0.55      2283

F1 Score :  51.70262252472292
[[ 211   27  155]
 [  21  100  526]
 [  32  178 1033]]


In [ ]:
import pickle
Pkl_Filename = "random_forest_text.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf, file)

In [ ]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier(loss='hinge',penalty='elasticnet',alpha=0.00001)
sgdc.fit(X_train,y_train)

SGDClassifier(alpha=1e-05, penalty='elasticnet')

In [ ]:
pred=sgdc.predict(X_test)
sgdc.score(X_test,y_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test,pred))
print("F1 Score : ",f1_score(y_test, pred, average='macro')*100)
print(confusion_matrix(y_test,pred))

Accuracy :  52.25580376697329
Report : 
               precision    recall  f1-score   support

          -1       0.57      0.64      0.60       374
           0       0.32      0.31      0.32       640
           1       0.61      0.59      0.60      1269

    accuracy                           0.52      2283
   macro avg       0.50      0.51      0.51      2283
weighted avg       0.52      0.52      0.52      2283

F1 Score :  50.72876860864827
[[238  37  99]
 [ 54 201 385]
 [122 393 754]]


In [ ]:
import pickle
Pkl_Filename = "sgdc_text.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(sgdc, file)

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier(criterion = 'gini')
dtc.fit(X_train,y_train)

DecisionTreeClassifier()

In [ ]:
pred=dtc.predict(X_test)
dtc.score(X_test,y_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test,pred))
print("F1 Score : ",f1_score(y_test, pred, average='macro')*100)
print(confusion_matrix(y_test,pred))

Accuracy :  51.68637757336837
Report : 
               precision    recall  f1-score   support

          -1       0.53      0.54      0.53       376
           0       0.29      0.24      0.26       651
           1       0.61      0.65      0.63      1256

    accuracy                           0.52      2283
   macro avg       0.47      0.48      0.48      2283
weighted avg       0.50      0.52      0.51      2283

F1 Score :  47.50421099808847
[[203  67 106]
 [ 67 157 427]
 [114 322 820]]


In [ ]:
import pickle
Pkl_Filename = "dtc_text.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(dtc, file)